In [ ]:
import os
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
!pip install tika

  Created wheel for tika: filename=tika-1.24-cp36-none-any.whl size=32885 sha256=9c29cd21fe95c99cfe18e7ff4c3925b59ed0a865df0898f479a8cb85bf9ecda3
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 2.7MB/s 
     |████████████████████████████████| 1.1MB 13.8MB/s 
     |████████████████████████████████| 3.0MB 13.3MB/s 
     |████████████████████████████████| 890kB 37.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=463f37273108b36693711c339e43a0cd3a7b7b8be92657bad268fc42a9e96eb3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from tika import parser
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("yuvraj/summarizer-cnndm")
cnn_model = AutoModelForSeq2SeqLM.from_pretrained("yuvraj/summarizer-cnndm")
cnn_model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 768, padding_idx=1)
      (layers): ModuleList(
        (0): EncoderLayer(
          (self_attn): SelfAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
 

In [ ]:
from google.colab import files
files.upload()

Saving article.pdf to article.pdf
Saving article.txt to article.txt


{'article.pdf': b'%PDF-1.5\n%\xc3\xa4\xc3\xbc\xc3\xb6\xc3\x9f\n2 0 obj\n<</Length 3 0 R/Filter/FlateDecode>>\nstream\nx\x9c\x95Z\xcb\x8e\xe3:\x0e\xdd\xd7Wd=\x8b\\\x89\xb2e\t\x18\x0c\x10\xdb\xc9\xfe\x02\r\xdc\x1f\x98\x070\x8b\x01\xe6n\xe6\xf7\x87<\xa4dIN9\xddhtU\xcaq,\x92:<<\xa4\xe2\xee\xfe\xf6\xbf\xaf\xff\xde\xdc\xcd\xdd\x1d\xa5[\xf4\xfe\x9e\x17\x7f[\xb2\xfe\xfe\xf3\x1f_\x7f\xfc\xe5\xf6\x1f\xbd\x83\xff\xfd\xf9\xaf\xaf\xf5\xc7\xd7\x1c\xef\xe9\xb6P\xbc\xe7\xdb\x8f\xbf\xdf~{\xf9\x9bw\xb7\x1f\xff\xfc\xab\xf3\x8e\\p\x93\x9b]t\x8bK\xfc:\xf3_\x0f\xb7\xba\xcd\xed\xfc3\xf2\xdfr\xf5\xc9W_\xdey\xef&O\x9e?\xe1\'?\xfb\xe8g\xdc1\xf9\xc5\'\x9f\xfd\x83_\xad\xfc\x7f\xf3;\xff|\xf2\x1d/r\xf6\xee\x83<\x91\x9b(\xe0\xa9\x13V\x98\xf8y\x1b?\xe5\xc5VD\x9a\xf8\xaa\xac\xc96\xd0L3\xbfOx\xfa\xf3o?\xfe\xfd\xf5\xfc\xf1\xf5\xfb[\xa7\xfc|\x8f\x83S\xf2\x88\x9d\x1fG0~w\x99\xcd\xe4ey\x99\tW\xa3,\xc0W\x08F\x88k\xb2\xcc\xea#\xdf=\x91\xfc\x8c\x8e\xf8\xaf\r\xae\xc3T\x82\xc9\xfc\x9cL\x93|\x96\x16\xbe3\xe1~s\x882\xdf5\xf3_\xec

In [ ]:
MAX_LEN = 1024
SUMM_LEN = 150
NUM_BEAMS = 4

In [ ]:
def summarize(art):
  cnn_model.eval()
  with torch.no_grad():
    source = tokenizer.batch_encode_plus([art], max_length=MAX_LEN, pad_to_max_length=True,return_tensors='pt', truncation='only_first')
    source_ids = source['input_ids']
    source_mask = source['attention_mask']
    ids = source_ids.to(device, dtype = torch.long)
    mask = source_mask.to(device, dtype = torch.long)

    cnn_generated_ids = cnn_model.generate(
        input_ids = ids,
        attention_mask = mask, 
        max_length=SUMM_LEN,
        num_beams=NUM_BEAMS,
        repetition_penalty=2.5,
        early_stopping=True
        )
    
    cnn_pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in cnn_generated_ids]
    return cnn_pred


In [ ]:
def chunkstring(string, length):
  string = " ".join(string.splitlines())
  chunks = []
  end_index = length-1
  start_index = 0
  while (True):
    if (end_index >= len(string)):
      chunks.append(string[start_index:])
      break
    while (string[end_index] != '.' and end_index > start_index):
      end_index = end_index - 1
    if (end_index == start_index):
      chunks.append(string[start_index : start_index+length+1])
      start_index = start_index+length
    elif (string[end_index] == '.'):
      chunks.append(string[start_index:end_index+1])
      start_index = end_index+1
    end_index = start_index + length - 1
  return chunks

In [ ]:
!ls

article.pdf  article.txt  sample_data


In [ ]:
doc_path = "article.pdf"

In [ ]:
def load_text(doc_path):
  text = ""
  doc_obj = open(doc_path, "r")
  ext = os.path.splitext(doc_path)[-1].lower()
  if (ext == ".pdf"):
    parsed = parser.from_file(doc_path)
    text = parsed["content"]
  elif (ext == ".txt"):
    text = doc_obj.read()
  return text

In [ ]:
def paragraphSummary(text):
  pSumm = []
  article = list(filter(lambda x : x != '', text.split('\n\n')))
  for art in article:
    summary = summarize(art)
    pSumm.append(summary[0])
  return pSumm

def fullSummary(text):
  final_summary = ""
  for chunk in chunkstring(text, 4000):
    final_summary = final_summary + summarize(chunk)[0]
  return final_summary

In [ ]:
text = load_text(doc_path)
text = text.strip()
# article = list(filter(lambda x : x != '', text.split('\n\n')))
print("Paragraph wise summary:")
pSumm = paragraphSummary(text)
print(pSumm)
# for art in article:
#   summary = summarize(art)
#   print(summary[0],"\n")
print("\n\nOverall summmary:")
full_summary = fullSummary(text)
print(full_summary)
# final_summary = ""
# for chunk in chunkstring(text, 1000):
#   final_summary = final_summary + summarize(chunk)[0]
# print(final_summary)

Paragraph wise summary:
[' Osbert Lancaster, CBE (4 August 1908 - 27 July 1986) was known for his cartoons in the British press.\nHe also had a lifelong work to inform the general public about good buildings and architectural heritage.', ' Lancaster published a series of books on architecture in the mid-1930s.\nLancaster was an undistinguished scholar at Charterhouse \nHis books have continued to be regarded as important works of reference on the subject.', ' Lancaster drew 10,000 "pocket cartoons" between 1939 and 1981.\nBetween 1939 and his retirement in 1981 he drew 10,000 of these "pocket cartoons"', ' Lancaster designed costumes and scenery for a new ballet.\nBetween then and the early 1970s he designed new productions for the Royal Ballet.\nIn his later years, his productivity declined when his health began to fail.', ' Lancaster was born in London in 1908.\nHis paternal grandfather, Sir William Lancaster, rose from modest beginnings to become the chief executive of the Prudentia

In [ ]:
print(full_summary)

 Osbert Lancaster, CBE (4 August 1908 - 27 July 1986) was known for his cartoons  in the British press.
Lancaster published the first of a series of books on architecture. was invited to contribute topical cartoons to The Daily Express.
Between 1939 and his retirement in 1981 he drew 10,000 of these "pocket cartoons" Lancaster died at his London home in 1975.
His diverse career, honoured by a knighthood in 1975, was celebrated by an exhibition at the Wallace Collection. Lancaster was an upper- middle class area in the early years of the 20th century.
The family maintained a staff of servants, including a cook and  nurse. Lancaster was a celebrated amateur footballer and occasional first class cricketer.
Lancaster's biographer Richard Boston says the school had an intellectual and aesthetic tradition. Lancaster shared Beerbohm's view that being an old boy was more pleasurable than being a pupil there.
Lancaster shared Beerbohm's view that being an old boy was more pleasurable than being